# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','m_total':'metros_total','m_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.columns

Index(['id', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [6]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1888 non-null   int64  
 1   categoria             1888 non-null   object 
 2   propiedad             1888 non-null   object 
 3   precio                1888 non-null   int64  
 4   metros_total          1888 non-null   int64  
 5   metros_construido     1888 non-null   int64  
 6   precio_m2_construido  1888 non-null   float64
 7   estacionamiento       1888 non-null   int64  
 8   recamaras             1888 non-null   int64  
 9   baño                  1888 non-null   int64  
 10  medio_baño            1888 non-null   int64  
 11  baño_total            1888 non-null   float64
dtypes: float64(2), int64(8), object(2)
memory usage: 177.1+ KB


In [7]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

df_total = df_total.drop(columns=['categoria'])
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [9]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)



print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)


# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    3.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      1.0        3.0    2.0
L                      4.0        5.0    5.0
S                      2.0        4.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    3.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      1.0        3.0    2.0
L                      4.0        5.0    5.0
S                      2.0        4.0    3.0
general                2.0        3.0    2.0


# Casa Mediana

In [10]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    3.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      1.0        3.0    2.0
E                      1.0        3.0    1.0
L                      2.0        5.0    4.0
S                      2.0        4.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 1          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    3.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      1.0        3.0    2.0
E                      1.0        3.0    1.0
L                      2.0        5.0    4.0
S                      2.0        4.0    3.0
general                1.0        3.0    2.0


# Precio Mercado

In [11]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3300000.0,3946465.0,3.569255e+06,3555000.0
1,B,2586000.0,3249000.0,2.882170e+06,2890000.0
2,C,1773000.0,2499000.0,2.071011e+06,1995000.0
3,D,1010000.0,1737200.0,1.294082e+06,1266000.0
4,E,346040.0,995000.0,6.866023e+05,630000.0
5,L,12260000.0,17500000.0,1.605000e+07,17000000.0
6,S,4130000.0,11083000.0,5.722591e+06,4810000.0


In [12]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,300,000.00
1     $2,586,000.00
2     $1,773,000.00
3     $1,010,000.00
4       $346,040.00
5    $12,260,000.00
6     $4,130,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,946,465.00
1     $3,249,000.00
2     $2,499,000.00
3     $1,737,200.00
4       $995,000.00
5    $17,500,000.00
6    $11,083,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descrip

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,300,000.00","$3,946,465.00","$3,569,254.94","$3,555,000.00"
1,B,"$2,586,000.00","$3,249,000.00","$2,882,169.75","$2,890,000.00"
2,C,"$1,773,000.00","$2,499,000.00","$2,071,010.57","$1,995,000.00"
3,D,"$1,010,000.00","$1,737,200.00","$1,294,081.87","$1,266,000.00"
4,E,"$346,040.00","$995,000.00","$686,602.33","$630,000.00"
5,L,"$12,260,000.00","$17,500,000.00","$16,050,000.00","$17,000,000.00"
6,S,"$4,130,000.00","$11,083,000.00","$5,722,591.43","$4,810,000.00"


In [13]:
todo_descripcion = df_total['precio'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $346,040.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $17,500,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will rai

,Mínimo,Máximo,Promedio,Mediana,segmento
precio,"$346,040.00","$17,500,000.00","$1,709,290.54","$1,095,000.00",general


In [14]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [15]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,300,000.00","$3,946,465.00","$3,569,254.94","$3,555,000.00"
1,B,"$2,586,000.00","$3,249,000.00","$2,882,169.75","$2,890,000.00"
2,C,"$1,773,000.00","$2,499,000.00","$2,071,010.57","$1,995,000.00"
3,D,"$1,010,000.00","$1,737,200.00","$1,294,081.87","$1,266,000.00"
4,E,"$346,040.00","$995,000.00","$686,602.33","$630,000.00"
5,L,"$12,260,000.00","$17,500,000.00","$16,050,000.00","$17,000,000.00"
6,S,"$4,130,000.00","$11,083,000.00","$5,722,591.43","$4,810,000.00"
7,general,"$346,040.00","$17,500,000.00","$1,709,290.54","$1,095,000.00"


# M2 Terreno

In [16]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3801508271.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    107.00 m²
1     70.00 m²
2     73.00 m²
3     67.00 m²
4     34.00 m²
5    220.00 m²
6    120.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3801508271.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    1,049.00 m²
1      265.00 m²
2      400.00 m²
3      404.00 m²
4      460.00 m²
5    1,972.00 m²
6    4,904.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,107.00 m²,"1,049.00 m²",255.61 m²,200.00 m²
1,B,70.00 m²,265.00 m²,173.92 m²,200.00 m²
2,C,73.00 m²,400.00 m²,164.99 m²,146.00 m²
3,D,67.00 m²,404.00 m²,136.97 m²,105.00 m²
4,E,34.00 m²,460.00 m²,129.45 m²,110.00 m²
5,L,220.00 m²,"1,972.00 m²","1,024.89 m²","1,000.00 m²"
6,S,120.00 m²,"4,904.00 m²",464.56 m²,228.00 m²


In [17]:
todo_descripcion = df_total[df_total['metros_total'] != 0]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3501913492.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3501913492.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    4,904.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3501913492.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will rai

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,34.00 m²,"4,904.00 m²",169.41 m²,120.00 m²,general


In [18]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [19]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,107.00 m²,"1,049.00 m²",255.61 m²,200.00 m²
1,B,70.00 m²,265.00 m²,173.92 m²,200.00 m²
2,C,73.00 m²,400.00 m²,164.99 m²,146.00 m²
3,D,67.00 m²,404.00 m²,136.97 m²,105.00 m²
4,E,34.00 m²,460.00 m²,129.45 m²,110.00 m²
5,L,220.00 m²,"1,972.00 m²","1,024.89 m²","1,000.00 m²"
6,S,120.00 m²,"4,904.00 m²",464.56 m²,228.00 m²
7,general,34.00 m²,"4,904.00 m²",169.41 m²,120.00 m²


# M2 Construcción

In [24]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    150.00 m²
1     95.00 m²
2     73.00 m²
3     60.00 m²
4     34.00 m²
5    275.00 m²
6    150.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    1,049.00 m²
1      600.00 m²
2      404.00 m²
3      420.00 m²
4      420.00 m²
5      598.00 m²
6    1,500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,150.00 m²,"1,049.00 m²",277.97 m²,250.00 m²
1,B,95.00 m²,600.00 m²,210.32 m²,159.00 m²
2,C,73.00 m²,404.00 m²,186.54 m²,159.00 m²
3,D,60.00 m²,420.00 m²,139.50 m²,120.00 m²
4,E,34.00 m²,420.00 m²,147.97 m²,110.00 m²
5,L,275.00 m²,598.00 m²,452.78 m²,500.00 m²
6,S,150.00 m²,"1,500.00 m²",314.99 m²,301.00 m²


In [25]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    4,904.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise 

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,34.00 m²,"4,904.00 m²",169.57 m²,120.00 m²,general


In [26]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [27]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,150.00 m²,"1,049.00 m²",277.97 m²,250.00 m²
1,B,95.00 m²,600.00 m²,210.32 m²,159.00 m²
2,C,73.00 m²,404.00 m²,186.54 m²,159.00 m²
3,D,60.00 m²,420.00 m²,139.50 m²,120.00 m²
4,E,34.00 m²,420.00 m²,147.97 m²,110.00 m²
5,L,275.00 m²,598.00 m²,452.78 m²,500.00 m²
6,S,150.00 m²,"1,500.00 m²",314.99 m²,301.00 m²
7,general,34.00 m²,"4,904.00 m²",169.57 m²,120.00 m²


# Pecio M2 Construcción

In [30]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3926392529.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,145.85
1     $5,333.33
2     $4,918.81
3     $2,607.14
4     $1,023.81
5    $27,591.97
6     $3,266.67
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\3926392529.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $26,309.77
1    $30,000.00
2    $28,273.97
3    $27,216.67
4    $16,634.62
5    $45,682.54
6    $49,250.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).app

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,145.85","$26,309.77","$14,548.69","$13,750.00"
1,B,"$5,333.33","$30,000.00","$16,469.35","$17,666.67"
2,C,"$4,918.81","$28,273.97","$12,724.30","$11,929.49"
3,D,"$2,607.14","$27,216.67","$10,934.14","$10,991.67"
4,E,"$1,023.81","$16,634.62","$6,638.38","$5,833.33"
5,L,"$27,591.97","$45,682.54","$36,897.54","$34,000.00"
6,S,"$3,266.67","$49,250.00","$20,994.58","$17,916.67"


In [32]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $1,023.81
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $49,250.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16276\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is depr

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,"$1,023.81","$49,250.00","$10,550.11","$10,285.71",general


In [33]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [34]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,145.85","$26,309.77","$14,548.69","$13,750.00"
1,B,"$5,333.33","$30,000.00","$16,469.35","$17,666.67"
2,C,"$4,918.81","$28,273.97","$12,724.30","$11,929.49"
3,D,"$2,607.14","$27,216.67","$10,934.14","$10,991.67"
4,E,"$1,023.81","$16,634.62","$6,638.38","$5,833.33"
5,L,"$27,591.97","$45,682.54","$36,897.54","$34,000.00"
6,S,"$3,266.67","$49,250.00","$20,994.58","$17,916.67"
7,general,"$1,023.81","$49,250.00","$10,550.11","$10,285.71"
